#### 데이터 분석

- 문제 정의 = 효율적인 광고집행을 해보자
- 가설 설정 = 제품 수요가 많은 지역을 안다면 효율적 광고 집행이 가능할 것 이다
- 데이터 분석 기본 세팅
- 데이터 분석
- 분석 결과 시각화
- 최종 결론
    - `서울 지역`에서 수강생의 숫자가 가장 많았고, 다음으로는 대전, 부산 지역 순으로 수강생이 많이 분포한 것을 알 수 있습니다!

#### 좋은 가설이란?

- 풀고자 하는 문제의 방향성과 일치하는 가설 → 무슨 문제를 풀고자 하는지 명확히 알 것
- 테스트 가능한 가설 → 데이터로 확인이 가능한 가설을 세울 것
- 액션으로 이어질 수 있는 가설 → 가설 검증 후 결과에 따라 특정 액션으로 유도될 수 있을 것

In [ ]:
# Pandas 라이브러리 불러오기

import pandas as pd

# 제품 수요 파일 불러오기

sparta_data = pd.read_table('../Data/students_area_detail.csv',sep=',')
sparta_data.head()

In [ ]:


sparta_data = pd.read_table('../Data/students_area_detail.csv',sep=',')
sparta_data.head()

In [ ]:
# 데이터 살펴 보기

sparta_data.head()

In [ ]:
# 지역 분류 하기
# set() 은 데이터의 중복없이 각각의 데이터가 unique한 값을 가질수 있게 하기 때문입니다!
# (즉, 같은 지역이 두번 출력 되는 것을 막을 수 있어요!😃 → 중복값 제거! 빠샤!)

category_range = set(sparta_data['area'])
print(category_range, len(category_range))


In [ ]:
# 접속지역, 해당 지역 위도, 해당 지역 경도 만으로 이루어진 테이블 만들기
# 새로운 테이블을 만들고자 할 땐 기존의 테이블에서 필요한 "열의 이름"을 대괄호에 넣어 변수에 지정해 주면 됩니다

area_info=sparta_data[['area','latitude','longitude']]

# 잘 만들어졌는지 초기 5개의 데이터 확인하기

area_info.head()

In [ ]:
# 중복되는 테이블의 정보를 삭제 시켜봅시다!
# drop_duplicates()을 이용하면, area(지역) 컬럼의 중복 데이터를 처리 할 수 있습니다. :)

area_info=area_info.drop_duplicates(['area'])
area_info

In [ ]:
#.reset_index()를 이용해, 인덱스를 재정렬 할 수 있어요!

area_info= area_info.reset_index()
area_info

In [ ]:
# 가공된 테이블인 “지역의 이름”을 기준으로 데이터를 정렬하기!

area_info = area_info.sort_values(by=["area"], ascending=[True])
area_info

In [ ]:
# 각 지역별 총 학생수 구하기

number_of_students = pd.DataFrame(sparta_data.groupby('area')['user_id'].count())
number_of_students

In [ ]:
# 지역별 총 학생수를 기존 테이블과 합치기
# merge()를 이용하여, 두 테이블을 병합 할수 있어요 :)!

result = pd.merge(area_info, number_of_students, on="area")
result

In [ ]:
# 시각화 라이브러리 불러오기

import matplotlib.pyplot as plt
import numpy as np

# 글씨 깨짐방지

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 폰트 설정

plt.rc('font', family='Malgun Gothic')

In [ ]:
# 지역별 수강생 수 라인 그래프 그리기
# 그래프 사이즈 변경

plt.figure(figsize=(10,5))

# 그래프 x축 y축

plt.plot(result['area'], result['user_id'])

# 그래프 명

plt.title('지역별 사용자 수')

# 그래프 x축 레이블

plt.xlabel('지역')

# 그래프 y축 레이블

plt.ylabel('사용자(명)')

# x축 눈금 수

plt.xticks(np.arange(13))

# 그래프 출력

plt.show()

In [ ]:
# 지도 라이브러리 불러오기
# 폴리움(Folium) = 분석한 데이터의 결과를 지도에 그리기 위한 라이브러리 입니다 😊
# 마커 클러스터(MarkerCluster) = 가까운 거리의 marker들을 군집시켜, 해당 건수를 표현해 줍니다.

import folium
from folium.plugins import MarkerCluster

In [ ]:
# 대한민국 위도 경도 설정 하기

m = folium.Map(location=[37.5536067,126.9674308],
               zoom_start=11) 
m

In [ ]:
# 지도에 수강생 분포 그려주기

for n in result.index:
  radius = result.loc[n,'user_id']
	# loc[n,"열 이름"]  => loc[]를 활용하여 n번째의 열을 조회 할수 있습니다!
	# 즉, n번째의 user의 수를 가져 올수 있는 것이죠!
  folium.CircleMarker([result['latitude'][n],result['longitude'][n]],
                        radius = radius/50, fill=True).add_to(m)
	# .add_to(m)를 활용하여, 지정해 두었던 우리나라의 지도를 가져올 수 있습니다!
m